### 01背包问题
有一个背包，它的容量为W。现在有n个物品，每个物品有重量$w[i]$和价值$v[i]$。
要求在不超过背包容量的前提下，选择一些物品放入背包，使得放入背包中的物品总价值最大。每个物品只能选择一次（0 - 1 表示不选或者选）。
1.	状态定义：
- 定义一个二维数组 dp[i][j]，表示在前 i 个物品中，背包容量为 j 时所能得到的最大价值。
2.	状态转移方程：
- 当不选第 i 个物品时： $dp[i][j]=dp[i-1][j]$
- 当选第 i 个物品时： $dp[i][j]=dp[i-1][j-w_i]+v_i$，前提是 $j>=w_i$
- 因此，状态转移方程为：
    $dp[i][j] = \max(dp[i-1][j], \  \  dp[i-1][j-w_i] + v_i)$

3.	初始条件：
- 当没有物品时，背包的最大价值为 0： $dp[0][j] = 0$，对于任意 j。
- 当背包容量为 0 时，无论有多少物品，最大价值也是 0：$dp[i][0] = 0$，对于任意 i。
4.	最终结果：
- 计算出 $dp[N][C]$ 即为最终答案，表示前 N 个物品在背包容量为 C 时的最大价值。

In [17]:
def knapsack_01(weights, values, capacity):
    n = len(weights)
    dp = [[0] * (capacity + 1) for _ in range(n + 1)]
    print("初始化dp:\n", dp)

    for i in range(1, n + 1):
        for j in range(1, capacity + 1):
            if j >= weights[i - 1]:
                dp[i][j] = max(dp[i - 1][j], dp[i - 1][j - weights[i - 1]] + values[i - 1])
            else:
                dp[i][j] = dp[i - 1][j]
    print(dp)
    return dp[n][capacity]

# 示例
weights = [1, 2, 3, 4]  # 每个物品的重量
values = [1, 6, 10, 16]   # 每个物品的价值
capacity = 5            # 背包容量
max_value = knapsack_01(weights, values, capacity)
print(f"最大价值是: {max_value}")

初始化dp:
 [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1], [0, 1, 6, 7, 7, 7], [0, 1, 6, 10, 11, 16], [0, 1, 6, 10, 16, 17]]
最大价值是: 17


In [7]:
from IPython.display import display
import pandas as pd


class KnapsackItem:
    def __init__(self, value, item_list):
        self.value = value
        self.item_list = item_list


def knapsack_01(items, capacity):
    n = len(items)
    dp = [[KnapsackItem(0, []) for _ in range(capacity + 1)] for _ in range(n + 1)]
    for i in range(1, n + 1):
        weight, value = items[i - 1]
        for j in range(1, capacity + 1):
            if j >= weight:
                if dp[i - 1][j].value > dp[i - 1][j - weight].value + value:
                    dp[i][j] = dp[i - 1][j]
                else:
                    new_item_list = dp[i - 1][j - weight].item_list.copy()
                    new_item_list.append(i - 1)
                    dp[i][j] = KnapsackItem(dp[i - 1][j - weight].value + value, new_item_list)
            else:
                dp[i][j] = dp[i - 1][j]
    return dp

#(weight, value)
items = [(1, 1), (2, 6), (3, 10), (4, 16)]
dp = knapsack_01(items, 5)
df_data = [[f"Value: {item.value}, List: {item.item_list}" for item in row] for row in dp]
df = pd.DataFrame(df_data)
styled_df = df.style.set_table_styles([
    {'selector': 'thead th', 'props': [('background-color', 'green')]},
    {'selector': 'tbody th', 'props': [('background-color', 'green')]}
])
display(styled_df)

,0,1,2,3,4,5
0,"Value: 0, List: []","Value: 0, List: []","Value: 0, List: []","Value: 0, List: []","Value: 0, List: []","Value: 0, List: []"
1,"Value: 0, List: []","Value: 1, List: [0]","Value: 1, List: [0]","Value: 1, List: [0]","Value: 1, List: [0]","Value: 1, List: [0]"
2,"Value: 0, List: []","Value: 1, List: [0]","Value: 6, List: [1]","Value: 7, List: [0, 1]","Value: 7, List: [0, 1]","Value: 7, List: [0, 1]"
3,"Value: 0, List: []","Value: 1, List: [0]","Value: 6, List: [1]","Value: 10, List: [2]","Value: 11, List: [0, 2]","Value: 16, List: [1, 2]"
4,"Value: 0, List: []","Value: 1, List: [0]","Value: 6, List: [1]","Value: 10, List: [2]","Value: 16, List: [3]","Value: 17, List: [0, 3]"


In [65]:
class Item:
    def __init__(self, weight, value):
        self.weight = weight
        self.value = value
        self.value_per_weight = value / weight

def greedy_knapsack(capacity, items):
    sorted_items = sorted(items, key=lambda item: item.value_per_weight, reverse=True)
    total_value = 0
    selected_items = []
    remaining_capacity = capacity
    for item in sorted_items:
        if item.weight <= remaining_capacity:
            total_value += item.value
            selected_items.append(item)
            remaining_capacity -= item.weight
    return total_value, selected_items

items = [Item(1, 1), Item(2, 6), Item(3, 10), Item(4, 16)]
capacity = 5
total_value, selected_items = greedy_knapsack(capacity, items)
print(f"Total value: {total_value}")
print("Selected items:")
for item in selected_items:
    print(f"Weight: {item.weight}, Value: {item.value}")

Total value: 17
Selected items:
Weight: 4, Value: 16
Weight: 1, Value: 1


贪心算法在01背包问题中通常基于价值密度（单位重量的价值）来选择物品。具体步骤如下：

1.	计算价值密度：对于每件物品，计算  $\text{density}[i] = \frac{v[i]}{w[i]} $。
2.	排序：按照价值密度从高到低对物品排序。
3.	选择物品：按照排序的顺序，选择物品放入背包，直到背包容量无法再放下更多的物品。

In [4]:
def knapsack_greedy(weights, values, capacity):
    # 计算价值密度并存储索引
    items = [(values[i] / weights[i], weights[i], values[i], i) for i in range(len(weights))]
    print(items)
    # 按价值密度降序排序
    items.sort(reverse=True, key=lambda x: x[0])
    
    total_value = 0
    chosen_items = []
    
    # 选择物品
    for density, weight, value, index in items:
        if capacity >= weight:
            capacity -= weight
            total_value += value
            chosen_items.append(index + 1)  # 存储选中的物品索引，+1是因为人们习惯从1开始计数
        if capacity == 0:
            break
    
    return total_value, chosen_items

# 物品的重量和价值
weights = [10, 20, 30]
values = [60, 100, 120]
capacity = 50

# 调用贪心算法
max_value, items_chosen = knapsack_greedy(weights, values, capacity)
print("最大价值:", max_value)
print("选择的物品:", items_chosen)

[(6.0, 10, 60, 0), (5.0, 20, 100, 1), (4.0, 30, 120, 2)]
最大价值: 160
选择的物品: [1, 2]


In [9]:
def knapsack_dynamic_programming(vals, costs, B):
    n = len(vals)
    V_star = max(vals)
    # 创建一个二维数组来存储中间结果
    S = [[-1]*(n*V_star + 1) for _ in range(n + 1)]
    # 初始化 S[1,p]
    for p in range(n*V_star + 1):
        if p == vals[0]:
            S[1][p] = [0]
        else:
            S[1][p] = -1
    # 动态规划填充 S[i,p]
    for i in range(1, n + 1):
        for p in range(n*V_star + 1):
            if i > 1:
                if vals[i - 1] <= p and S[i - 1][p - vals[i - 1]]!= -1:
                    option1 = costs[S[i - 1][p][0]] if S[i - 1][p]!= -1 else float('inf')
                    option2 = costs[i - 1] + costs[S[i - 1][p - vals[i - 1]][0]] if S[i - 1][p - vals[i - 1]]!= -1 else float('inf')
                    if option1 < option2:
                        S[i][p] = S[i - 1][p]
                    else:
                        S[i][p] = [i - 1] + S[i - 1][p - vals[i - 1]]
                else:
                    S[i][p] = S[i - 1][p]
    # 找到最优解
    max_value = 0
    optimal_subset = None
    for p in range(n*V_star + 1):
        if S[n][p]!= -1 and costs[S[n][p][0]] <= B:
            total_value = sum([vals[idx] for idx in S[n][p]])
            if total_value > max_value:
                max_value = total_value
                optimal_subset = S[n][p]
    return max_value, optimal_subset

# 示例用法

#items = [(1, 1), (2, 6), (3, 10), (4, 16)]

values = [1, 6, 10, 16]
costs = [1, 2, 3, 4]
capacity = 5
v, subset = knapsack_dynamic_programming(values, costs, capacity)
print(v)
print("最优子集的索引：", subset)

33
最优子集的索引： [3, 2, 1, 0]
